In [13]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread
app = Flask(__name__)
CORS(app)
posts = []

@app.route('/posts', methods=['GET'])
def get_posts():
    return jsonify(posts)

@app.route('/posts', methods=['POST'])
def add_post():
    print("hello")
    post_data = request.json
    posts.append(post_data)
    return jsonify(post_data), 201

from werkzeug.serving import run_simple

def run_app():
    run_simple('localhost', 5000, app)

# Then start the thread as before
flask_thread = Thread(target=run_app)
flask_thread.start()

Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> General -> AirDrop & Handoff.


In [14]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPEN_AI_KEY")

Get video transcript or audio file if video has no captions

In [16]:
import glob
from datetime import datetime
def get_todays_date():
    # Get today's date
    today = datetime.now()
    # Format the date as a string (e.g., "2024-03-14")
    date_string = today.strftime("%Y-%m-%d")
    return date_string
from pytube import YouTube
from pytube import Playlist
from youtube_transcript_api import YouTubeTranscriptApi
from datetime import date

def create_transcripts(type, dated):
    if (type == 1):
        videos = open("inputs.txt", "r")
        video_obj = {}
        directory_path = 'Transcripts'

        # Get a list of all .txt file paths in the directory
        txt_file_paths = glob.glob(os.path.join(directory_path, '*.txt'))

        # Loop through the list and remove each .txt file
        for file_path in txt_file_paths:
            try:
                os.remove(file_path)
                print(f"Deleted {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")
        for video in videos:
            # Get YouTube video
            yt = YouTube(video)
            video_obj[yt.title] = { "author": yt.author, "transcribed_on": date.today() }
            caption=YouTubeTranscriptApi.get_transcript(yt.video_id, languages=['en'])
            # Get video transcription, if none exist, download audio (last stream is guaranteed English)
            if not caption:
                streams = yt.streams.filter(only_audio=True, file_extension="mp4")
                streams[-1].download("audio_files")
            else:
                f = open("transcripts/" + yt.title + ".txt", "w")
                for obj in caption:
                    f.write(obj['text'])
                f.close()
            description = yt.description
            if not os.path.isdir("key info/" + video_obj[yt.title]['author']):
                        os.mkdir("key info/" + video_obj[yt.title]['author'])
            print("hello")           
            t = open("key info/" + video_obj[yt.title]['author'] + "/" + yt.title + ".txt", "w")
            if description is not None:
                t.write("Title:" + yt.title  + "\nAuthor: Lev Pollock\nDate: "+ get_todays_date() + "\nChannel: " + video_obj[yt.title]['author'] + "\nDescription: "+ description + "\n")
            else:
                t.write("Title:" + yt.title  + "\nAuthor: Lev Pollock\nDate: "+ get_todays_date() + "\nChannel: " + video_obj[yt.title]['author']+ "\n")
            t.close()
    else :
        playlist = open("channels.txt", "r")
        
        video_obj = {}
        directory_path = 'Transcripts'

        # Get a list of all .txt file paths in the directory
        txt_file_paths = glob.glob(os.path.join(directory_path, '*.txt'))

        # Loop through the list and remove each .txt file
        for file_path in txt_file_paths:
            try:
                os.remove(file_path)
                print(f"Deleted {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")
        for channel in playlist:
            print(channel)
            c = Playlist(channel)
            for url in c.video_urls:
                print(url)
create_transcripts(1,0)

hello
hello
hello


Transcribe audio files (if they exist)

In [ ]:
import subprocess
import sys

audio_files = os.fsencode("audio_files")

for file in os.listdir(audio_files):
    filename = os.fsdecode(file)
    if filename.endswith(".mp4"):
        title = filename.replace(".mp4", "")

        # convert mp4 file to wav (16-bit)
        new_filename = title + ".wav"
        subprocess.run([
            'ffmpeg',
            '-i', os.path.join("audio_files", filename),
            '-ar', '16000',
            '-ac', '1',
            '-c:a', 'pcm_s16le',
            os.path.join("audio_files", new_filename)
        ])

        # call local whisper model
        result = subprocess.check_output([
            './main',
            '-f', '../Podcast-Summarizer/audio_files/' + new_filename
        ], cwd="../whisper.cpp")

        f = open("transcripts/" + title + ".txt", "w")
        f.write(result.decode(sys.stdout.encoding).strip())
        f.close()

print(video_obj)        

NameError: name 'video_obj' is not defined

Summarize Transcripts

In [ ]:
# Path to your 'transcripts' directory
directory_path = 'transcripts'

# Iterate over each file in the directory

for filename in os.listdir(directory_path):
    final_string = ""
    # Check if the file is a .txt file
    if filename.endswith('.txt'):
        # Construct the full path to the file
        file_path = os.path.join(directory_path, filename)
        file_path2 = os.path.join('other', 'example.txt')
        # Open and read the file
        example_file=open(file_path2, 'r')
        gpt4 = 1
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            if (gpt4==1):
                part = openai.chat.completions.create(model="gpt-4-turbo-preview", 
                    messages=[
                    {"role": "system", "content": "You are an investment analyst who has been tasked with summarizing podcast transcripts for your boss so he can make strategic investment decisions."},
                    {"role": "user", "content": "Please summarize the following podcast transcript for your boss in as much detail as possible. you must focus on the key points discussed, including the main arguments, insights, and takeaways. Additionally, identify any interesting quotes or counter-arguments presented. Do not miss a single point discussed in the transcript"},
                    {"role": "user", "content": content},
                    {"role": "user", "content": "there should be two sections in the output, the high level takeaways of each subject which includes key quotes, and then a detailed breakdown section as noted by the next prompt which dictates the schema of how the answer must be, try to stay away from generally saying what was talked about and instead focus on the specific details mentioned with some quotes and further information"},
               #  {"role": "user", "content": "it should include a section of high level key takeaways, some of which are crucial quotes but also include some specific takeaways from the whole thing - approximately 10 of them, then a section of a deep breakdown of the entire transcript that includes a header for each subject that is talked about in the podcast and include 5 detailed bullets for each subject, try to stay away from generally saying what was talked about and instead focus on the specific details mentioned with some quotes and further information"},
                    {"role": "user", "content": combined_message},
                    {"role": "user", "content": "you must ensure all topics are covered comprehensively according to the schema."}
                ]
                )
                #print(first_part)
                #print("part done")
                final_string = final_string + part.choices[0].message.content + "\n"
            else:
                string_length = len(content)
                multiple = string_length // 30000 + 1
                div = string_length // multiple + multiple
                print(div)
                print(multiple)
                first_part = content[:div]
                second_part = content[div:]
                full_string = ""
                for x in range(1, multiple+1): 
                    print(x)
                    combined_message = "you must use the following as a schema for how you structure the output" + "\n" + example_file.read()
                    #Split the string into two halves
                    part = openai.chat.completions.create(model="gpt-3.5-turbo", 
                        messages=[
                        {"role": "system", "content": "You are an investment analyst who has been tasked with summarizing podcast transcripts for your boss so he can make strategic investment insights."},
                        {"role": "user", "content": "Please summarize the following podcast transcript for your boss in as much detail as possible. you must focus on the key points discussed, including the main arguments, insights, and takeaways. Additionally, identify any interesting quotes or counter-arguments presented. Please present in very detailed bullet point form, do not miss a single point discussed in the transcript"},
                        {"role": "user", "content": first_part},
                        {"role": "user", "content": "there should be two sections in the output, the high level takeaways of each subject which includes key quotes, and then a detailed breakdown section as noted by the next prompt, try to stay away from generally saying what was talked about and instead focus on the specific details mentioned with some quotes and further information"},
                    #  {"role": "user", "content": "it should include a section of high level key takeaways, some of which are crucial quotes but also include some specific takeaways from the whole thing - approximately 10 of them, then a section of a deep breakdown of the entire transcript that includes a header for each subject that is talked about in the podcast and include 5 detailed bullets for each subject, try to stay away from generally saying what was talked about and instead focus on the specific details mentioned with some quotes and further information"},
                        {"role": "user", "content": combined_message},
                        {"role": "user", "content": "you must ensure all topics are covered comprehensively according to the schema."}
                    ]
                    )
                    full_string = full_string + "\n" + part.choices[0].message.content
                    #print(part.choices[0].message.content)
                    if (x!=1):
                        new_part = openai.chat.completions.create(model="gpt-3.5-turbo", 
                        messages=[
                        {"role": "system", "content": "You are an investment analyst who has been tasked with summarizing podcast transcripts for your boss so he can make strategic investment insights."},
                        {"role": "user", "content": "Please combine the following two parts of a podcast summary into a single, cohesive document. You must ensure that the final summary is structured according to the specified schema with all relevant details maintained and properly integrated. The schema must be as follows:"},
                        {"role": "user", "content": combined_message},
                        {"role": "user", "content": "Here are the parts to combine, they are in the schema format as well:"},
                    #  {"role": "user", "content": "it should include a section of high level key takeaways, some of which are crucial quotes but also include some specific takeaways from the whole thing - approximately 10 of them, then a section of a deep breakdown of the entire transcript that includes a header for each subject that is talked about in the podcast and include 5 detailed bullets for each subject, try to stay away from generally saying what was talked about and instead focus on the specific details mentioned with some quotes and further information"},
                        {"role": "user", "content": "part 1: " + part.choices[0].message.content},
                        {"role": "user", "content": "part 2: " + final_string},
                        {"role": "user", "content": "you must ensure all topics are covered comprehensively according to the schema and that you maintain the two components of key highlights and detailed breakdown."}
                        ]
                        )
                    #print(first_part)
                    #print("part done")
                        final_string = new_part.choices[0].message.content
                    else:
                        final_string = part.choices[0].message.content
                    first_part = second_part[:div]
                    second_part = second_part[div:]
            summary = openai.chat.completions.create(model="gpt-3.5-turbo", 
                        messages=[
                            {"role": "system", "content": "You are an investment analyst who has been tasked with summarizing podcast transcripts for your boss so he can make strategic investment insights."},
                            {"role": "user", "content": "Write a 100 word summary of this summary of a podcast"},
                            {"role": "user", "content": final_string}
                        ]
            )
            final_string = "Summary: " + summary.choices[0].message.content + "\n" + final_string 
            title = filename.split('.')[0]
            if not os.path.isdir("summaries/" + video_obj[title]['author']):
                os.mkdir("summaries/" + video_obj[title]['author'])
            if not os.path.isdir("unorganized summaries/" + video_obj[title]['author']):
                os.mkdir("unorganized summaries/" + video_obj[title]['author'])
                
            f = open("summaries/" + video_obj[title]['author'] + "/" + filename, "w")
            f.write(final_string)
            f.close()
            t = open("unorganized summaries/" + video_obj[title]['author'] + "/" + filename, "w")
            t.write(full_string)
            t.close()

NameError: name 'combined_message' is not defined

In [ ]:
from docx import Document



import os

# Define the base directories
base_dir2 = 'Summaries'
base_dir1 = 'key info'
output_base_dir = 'Word Files'

# Ensure the output base directory exists
os.makedirs(output_base_dir, exist_ok=True)
# Walk through all subdirectories and files in the first base directory
for subdir, dirs, files in os.walk(base_dir1):
    for filename in files:
        # Construct the relative path
        rel_path = os.path.relpath(subdir, base_dir1)
        
        # Check if the corresponding subdirectory exists in the second base directory
        corresponding_subdir = os.path.join(base_dir2, rel_path)
        if not os.path.exists(corresponding_subdir):
            print(f"Skipping {rel_path}: corresponding subdirectory does not exist in the second base directory.")
            continue  # Skip to the next file

        # Construct the full paths for both source files
        file_path1 = os.path.join(subdir, filename)
        file_path2 = os.path.join(corresponding_subdir, filename)

        # Check if the corresponding file exists in the second directory
        if os.path.exists(file_path2):
            doc = Document()

            # Read and add the contents of the first file
            with open(file_path1, 'r', encoding='utf-8') as file1:
                content1 = file1.read()
                doc.add_paragraph(content1)

            # Read and add the contents of the second file
            with open(file_path2, 'r', encoding='utf-8') as file2:
                content2 = file2.read()
                doc.add_paragraph(content2)

            # Construct the output directory path and ensure it exists
            output_dir = os.path.join(output_base_dir, rel_path)
            os.makedirs(output_dir, exist_ok=True)

            # Construct the output file path and save the combined content as a Word document
            output_file_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.docx')
            doc.save(output_file_path)
        else:
            print(f"File {filename} does not exist in both directories under {rel_path}.")
base_dir2 = 'unorganized Summaries'
base_dir1 = 'key info'
output_base_dir = 'Word Files'

# Ensure the output base directory exists
os.makedirs(output_base_dir, exist_ok=True)
# Walk through all subdirectories and files in the first base directory
for subdir, dirs, files in os.walk(base_dir1):
    for filename in files:
        # Construct the relative path
        rel_path = os.path.relpath(subdir, base_dir1)
        
        # Check if the corresponding subdirectory exists in the second base directory
        corresponding_subdir = os.path.join(base_dir2, rel_path)
        if not os.path.exists(corresponding_subdir):
            print(f"Skipping {rel_path}: corresponding subdirectory does not exist in the second base directory.")
            continue  # Skip to the next file

        # Construct the full paths for both source files
        file_path1 = os.path.join(subdir, filename)
        file_path2 = os.path.join(corresponding_subdir, filename)

        # Check if the corresponding file exists in the second directory
        if os.path.exists(file_path2):
            doc = Document()

            # Read and add the contents of the first file
            with open(file_path1, 'r', encoding='utf-8') as file1:
                content1 = file1.read()
                doc.add_paragraph(content1)

            # Read and add the contents of the second file
            with open(file_path2, 'r', encoding='utf-8') as file2:
                content2 = file2.read()
                doc.add_paragraph(content2)

            # Construct the output directory path and ensure it exists
            output_dir = os.path.join(output_base_dir, rel_path)
            os.makedirs(output_dir, exist_ok=True)

            # Construct the output file path and save the combined content as a Word document
            output_file_path = os.path.join(output_dir, os.path.splitext(filename)[0] +' unorganized.docx')
            doc.save(output_file_path)
        else:
            print(f"File {filename} does not exist in both directories under {rel_path}.")

# Notify the user that the process is complete
print("All applicable files have been successfully combined and saved.")

        


Skipping Capital Allocators with Ted Seides: corresponding subdirectory does not exist in the second base directory.
Skipping Capital Allocators with Ted Seides: corresponding subdirectory does not exist in the second base directory.
Skipping David Rubenstein: corresponding subdirectory does not exist in the second base directory.
File 2024 Macro Outlook Not Rosy - Expert Felix Zulauf Explains.txt does not exist in both directories under The Meb Faber Show Podcast.
File BlackRock's Bond Chief: Lock That Bond Yield! | Rick Rieder.txt does not exist in both directories under The Meb Faber Show Podcast.
Skipping Comic Doctor: corresponding subdirectory does not exist in the second base directory.
Skipping Comic Doctor: corresponding subdirectory does not exist in the second base directory.
Skipping FORMULA 1: corresponding subdirectory does not exist in the second base directory.
Skipping FORMULA 1: corresponding subdirectory does not exist in the second base directory.
Skipping FORMULA 1